<a href="https://www.kaggle.com/code/artemsilvander/learning-transformers-with-hugging-face?scriptVersionId=122292642" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This notebook is created to learn and practice Transformers library from Hugging Face.

Feel free to comment and share your ideas.

Please upvote if this notebook helped you. Thanks!

In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

# Pre-built pipelines

In [2]:
# transformers library has various pre-biult pipelines for various NLP and CV tasks
from transformers.pipelines import PIPELINE_REGISTRY

#Get the list of tasks that are supported by Huggingface pipeline

pip_list = PIPELINE_REGISTRY.get_supported_tasks()
for i in pip_list: print(i)

audio-classification
automatic-speech-recognition
conversational
depth-estimation
document-question-answering
feature-extraction
fill-mask
image-classification
image-segmentation
image-to-text
ner
object-detection
question-answering
sentiment-analysis
summarization
table-question-answering
text-classification
text-generation
text2text-generation
token-classification
translation
video-classification
visual-question-answering
vqa
zero-shot-classification
zero-shot-image-classification
zero-shot-object-detection


In [3]:
#Get information about a specific task, pt - PyTorch, tf - TensorFlow
print("\nDefault Model for Sentiment Analysis: ")
print(PIPELINE_REGISTRY.check_task('sentiment-analysis')[1].get('default'))


Default Model for Sentiment Analysis: 
{'model': {'pt': ('distilbert-base-uncased-finetuned-sst-2-english', 'af0f99b'), 'tf': ('distilbert-base-uncased-finetuned-sst-2-english', 'af0f99b')}}


# Sentiment Analysis

In [4]:
from transformers import pipeline
import os

#Load a pipeline. This will download the model checkpoint from huggingface and cache it 
#locally on disk. If model is already available in cache, it will simply use the cached version
#Download will usually take a long time, depending on network bandwidth

sentiment_classifier = pipeline("sentiment-analysis")

#Cache usually available at : <<user-home>>.cache\huggingface\hub

cache_dir = os.path.expanduser('~') + "/.cache/huggingface/hub"
print("Huggingface Cache directory is : ", cache_dir)

#Contents of cache directory
os.listdir(cache_dir)

Huggingface Cache directory is :  /root/.cache/huggingface/hub


['version.txt', 'models--distilbert-base-uncased-finetuned-sst-2-english']

In [5]:
#Predict sentiment using the pipeline
sentiment_results=sentiment_classifier("This is a great course")
print(sentiment_results)

[{'label': 'POSITIVE', 'score': 0.9998713731765747}]


In [6]:
#A second example
sentiment_results=sentiment_classifier("The download speed is really bad")
print(sentiment_results)

[{'label': 'NEGATIVE', 'score': 0.9998070597648621}]


In [7]:
# using a specific model in the pipeline
sentiment_classifier = pipeline(task="sentiment-analysis",
                                model="finiteautomata/bertweet-base-sentiment-analysis")

sentiment_result=sentiment_classifier("This is a great course")

print(sentiment_result)

#Contents of cache directory
os.listdir(cache_dir)

[{'label': 'POS', 'score': 0.9920700192451477}]


['models--finiteautomata--bertweet-base-sentiment-analysis',
 'version.txt',
 'models--distilbert-base-uncased-finetuned-sst-2-english']

# Named Entity Recognition

In [8]:
# extract NERs from text (person, company, location, date, custom, order_number, etc)
from transformers import pipeline

input_text="Sam went to California on the 23rd of August. \
There, he visited Google headquarters with John Smith and bought a cap for $23"

basic_ner = pipeline("ner")

basic_ner(input_text)

[{'entity': 'I-PER',
  'score': 0.99887806,
  'index': 1,
  'word': 'Sam',
  'start': 0,
  'end': 3},
 {'entity': 'I-LOC',
  'score': 0.99972683,
  'index': 4,
  'word': 'California',
  'start': 12,
  'end': 22},
 {'entity': 'I-ORG',
  'score': 0.9960085,
  'index': 15,
  'word': 'Google',
  'start': 64,
  'end': 70},
 {'entity': 'I-PER',
  'score': 0.99891376,
  'index': 18,
  'word': 'John',
  'start': 89,
  'end': 93},
 {'entity': 'I-PER',
  'score': 0.99921584,
  'index': 19,
  'word': 'Smith',
  'start': 94,
  'end': 99}]

In [9]:
#Print model architecture
print(basic_ner.model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-1

In [10]:
# print model configuration
print(basic_ner.model.config)

BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_s

In [11]:
# Using a Custom Model and tokenizer
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates", 
                                          from_pt=True)

model = TFAutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates",
                                                          from_pt=True)

print(model.config.id2label)

{0: 'O', 1: 'I-LOC', 2: 'I-PER', 3: 'I-MISC', 4: 'I-ORG', 5: 'I-DATE'}


In [12]:
#Prediction
enhanced_ner = pipeline('ner', 
                        model=model, 
                        tokenizer=tokenizer, 
                        aggregation_strategy="simple")
enhanced_ner(input_text)

[{'entity_group': 'PER',
  'score': 0.9776213,
  'word': 'Sam',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9936407,
  'word': 'California',
  'start': 11,
  'end': 22},
 {'entity_group': 'DATE',
  'score': 0.9235597,
  'word': 'August',
  'start': 37,
  'end': 44},
 {'entity_group': 'ORG',
  'score': 0.5721672,
  'word': 'Google',
  'start': 63,
  'end': 70},
 {'entity_group': 'PER',
  'score': 0.9938346,
  'word': 'John Smith',
  'start': 88,
  'end': 99},
 {'entity_group': 'DATE',
  'score': 0.6406405,
  'word': '23',
  'start': 122,
  'end': 124}]

# Question answering

In [13]:
# importing answering pipeline from Transformers and pre-trained model deepset/minilm-uncased-squad2
# model is trained on domain-specific limited text, fast and accurate

from transformers import pipeline

# providing context for the model
context="""
Earth is the third planet from the Sun and the only astronomical object 
known to harbor life. While large volumes of water can be found 
throughout the Solar System, only Earth sustains liquid surface water. 
About 71% of Earth's surface is made up of the ocean, dwarfing 
Earth's polar ice, lakes, and rivers. The remaining 29% of Earth's 
surface is land, consisting of continents and islands. 
Earth's surface layer is formed of several slowly moving tectonic plates, 
interacting to produce mountain ranges, volcanoes, and earthquakes. 
Earth's liquid outer core generates the magnetic field that shapes Earth's 
magnetosphere, deflecting destructive solar winds.
"""

# activating pipelone
quan_pipeline = pipeline("question-answering", 
                         model="deepset/minilm-uncased-squad2")

# generating answer for our question, based on our context
answer=quan_pipeline(question="How much of earth is land?",
             context=context)
print(answer)

{'score': 0.9553400874137878, 'start': 327, 'end': 330, 'answer': '29%'}


In [14]:
# low confidence answer, we can filter only high confidence answers for our task
print("\nAnother question :")
print(quan_pipeline( question="How are mountain ranges created?",
             context=context))


Another question :
{'score': 0.26154088973999023, 'start': 399, 'end': 471, 'answer': "Earth's surface layer is formed of several slowly moving tectonic plates"}


In [15]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00


In [16]:
# the SQuAD metric - to evaluate QA models (predicted vs correct answer) using multiple metrics

from evaluate import load
squad_metric = load("squad_v2")

#Ignoring Context & Question as they are not needed for evaluation
#This example is to showcase how the evaluation works based on match between the prediction
#and the correct answer

correct_answer="Paris"

predicted_answers=["Paris",
                 "London",
                 "Paris is one of the best cities in the world"]

cum_predictions=[]
cum_references=[]

# generate predicted answers score vs our correct answer
for i in range(len(predicted_answers)):
    
    #Use the input format for predictions
    predictions = [{'prediction_text':predicted_answers[i], 
                    'id': str(i),
                    'no_answer_probability': 0.}]
    cum_predictions.append(predictions[0])
    
    #Use the input format for answers
    references = [{'answers': {'answer_start': [1], 
                               'text': [correct_answer]}, 
                   'id': str(i)}]
    cum_references.append(references[0])

    # return the evaluation of our answers
    results = squad_metric.compute(predictions=predictions,
                                   references=references)
    print("F1 is", results.get('f1'), 
          " for answer :", predicted_answers[i])
    
#Compute for cumulative Results, count of answers, etc
cum_results=squad_metric.compute(predictions=cum_predictions,
                                 references=cum_references)
print("\n Cumulative Results : \n",cum_results)

F1 is 100.0  for answer : Paris
F1 is 0.0  for answer : London
F1 is 22.22222222222222  for answer : Paris is one of the best cities in the world

 Cumulative Results : 
 {'exact': 33.333333333333336, 'f1': 40.74074074074074, 'total': 3, 'HasAns_exact': 33.333333333333336, 'HasAns_f1': 40.74074074074074, 'HasAns_total': 3, 'best_exact': 33.333333333333336, 'best_exact_thresh': 0.0, 'best_f1': 40.74074074074074, 'best_f1_thresh': 0.0}


# Text summarization

In [17]:
verbose_text ="""
Earth is the third planet from the Sun and the only astronomical object 
known to harbor life. 
While large volumes of water can be found 
throughout the Solar System, only Earth sustains liquid surface water. 
About 71% of Earth's surface is made up of the ocean, dwarfing 
Earth's polar ice, lakes, and rivers. 
The remaining 29% of Earth's 
surface is land, consisting of continents and islands. 
Earth's surface layer is formed of several slowly moving tectonic plates, 
interacting to produce mountain ranges, volcanoes, and earthquakes. 
Earth's liquid outer core generates the magnetic field that shapes Earth's 
magnetosphere, deflecting destructive solar winds.
"""

verbose_text = verbose_text.replace("\n","")

In [18]:
# selecting summarization pipeline
from transformers import pipeline


extractive_summarizer = pipeline("summarization", 
                                 min_length=10, 
                                 max_length=100)

#Extractive summarization
extractive_summary=extractive_summarizer(verbose_text)

print(extractive_summary[0].get("summary_text"))


 Earth is the third planet from the Sun and the only astronomical object known to harbor life . About 71% of Earth's surface is made up of the ocean, dwarfing Earth's polar ice, lakes, and rivers . The remaining 29% of the surface is land, consisting of continents and islands .


In [19]:
print("Checkpoint used: ", extractive_summarizer.model.config)

Checkpoint used:  BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_en

In [20]:
!pip install evaluate

In [21]:
!pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=b842d42be7bc679e09cd184fee55425e2378c4024fd2a8bbdedfeed8412508e1
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge_score


In [22]:
# evaluate with ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
import evaluate

rouge_evaluator = evaluate.load("rouge")

#Evaluate exact match strings
reference_text=["This is the same string"]
predict_text=["This is the same string"]

eval_results=rouge_evaluator.compute(predictions=predict_text, 
                                     references=reference_text)
print("Results for Exact match",eval_results)

Results for Exact match {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [23]:
#Evaluate no-match strings
reference_text=["This is the different string"]
predict_text=["Google can predict warm weather"]

eval_results=rouge_evaluator.compute(predictions=predict_text, 
                                     references=reference_text)
print("\nResults for no match", eval_results)


Results for no match {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [24]:
#Evaluate summary
eval_results=rouge_evaluator.compute(
    predictions=[extractive_summary[0].get("summary_text")], 
    references=[verbose_text])

print("\nResults for Summary generated", eval_results)


Results for Summary generated {'rouge1': 0.6369426751592356, 'rouge2': 0.5935483870967743, 'rougeL': 0.624203821656051, 'rougeLsum': 0.624203821656051}


# Natural Language Generation

In [25]:
# cintent creation
from transformers import pipeline

text_generator = pipeline("text-generation", 
                          model="gpt2")
transformers.set_seed(1)

input_text="Natural Language Processing is a \
growing domain in machine learning"

synthetic_text=text_generator(input_text,
                              num_return_sequences=3,
                              max_new_tokens=50)

for text in synthetic_text:
    print(text.get("generated_text") ,"\n-----------------")

/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1187: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


Natural Language Processing is a growing domain in machine learning applications. The purpose of this project is to provide a scalable way to develop a variety of languages on top of neural net technologies. Some applications are being developed that use the underlying neural network and can easily integrate with existing applications. These are shown below demonstrating 
-----------------
Natural Language Processing is a growing domain in machine learning and there are numerous applications available in the field of AI in general. With the current state of AI research in AI, we are exploring these topics, including:

Using AI for Machine Learning Science/Sustainability

Using Machine Learning in 
-----------------
Natural Language Processing is a growing domain in machine learning. This means that every new task, even one you've never thought to create, may be used as a foundation for generating a more comprehensive set of AI-based machine learning algorithms. But the problem of human

# Bot conversation

In [26]:
# chatbot conversation example 
from transformers import  Conversation

conversational_pipeline = pipeline("conversational", 
                                   model="facebook/blenderbot_small-90M")

print(conversational_pipeline.model.config)

BlenderbotSmallConfig {
  "_name_or_path": "facebook/blenderbot_small-90M",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BlenderbotSmallForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 8,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 8,
  "eos_token_id": 2,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id

In [27]:
#Sample inputs
first_input="Do you have any hobbies?"
second_input = "I like to watch movies"
third_input = "action movies"

#Create a context
bot_conversation = Conversation(first_input)

print("\nFirst Exchange: \n--------------------")

conversational_pipeline(bot_conversation)
print(" User Input:", bot_conversation.past_user_inputs[0])
print(" Bot Output:", bot_conversation.generated_responses[0])

print("\nSecond Exchange: \n--------------------")
bot_conversation.add_user_input(second_input)
conversational_pipeline(bot_conversation)

print(" User Input:", bot_conversation.past_user_inputs[1])
print(" Bot Output:", bot_conversation.generated_responses[1])

print("\nThird Exchange: \n--------------------")
bot_conversation.add_user_input(third_input)
conversational_pipeline(bot_conversation)

print(" User Input:", bot_conversation.past_user_inputs[2])
print(" Bot Output:", bot_conversation.generated_responses[1])

print("\nAccessing All Responses: ")
print(bot_conversation)


First Exchange: 
--------------------
 User Input: Do you have any hobbies?
 Bot Output: yes, i love going to the beach. what about you? do you have any hobbies?

Second Exchange: 
--------------------
 User Input: I like to watch movies
 Bot Output: i love going to the beach. i also like to watch movies. what kind of movies do you like?

Third Exchange: 
--------------------
 User Input: action movies
 Bot Output: i love going to the beach. i also like to watch movies. what kind of movies do you like?

Accessing All Responses: 
Conversation id: 9c4d3df9-d525-4d8d-9517-4660b1589bc4 
user >> Do you have any hobbies? 
bot >> yes, i love going to the beach. what about you? do you have any hobbies? 
user >> I like to watch movies 
bot >> i love going to the beach. i also like to watch movies. what kind of movies do you like? 
user >> action movies 
bot >> i love going to the beach as well. i like action movies as well, but i don't get to see them often. 



# Translation with Hugging Face

In [28]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

source_english="Acme is a technology company based in New York and Paris"

inputs_german = tokenizer(
    "translate English to German: " + source_english,
    return_tensors="pt",
)
outputs_german = model.generate(
    inputs_german["input_ids"], 
    max_length=40)

print("German Translation: ",
      tokenizer.decode(outputs_german[0], 
                       skip_special_tokens=True))

inputs_french = tokenizer(
    "translate English to French: " + source_english, 
    return_tensors="pt",
)
outputs_french = model.generate(
    inputs_french["input_ids"], 
    max_length=40)

print("French Translation: ", 
      tokenizer.decode(outputs_french[0], 
                       skip_special_tokens=True))

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


German Translation:  Acme ist ein Technologieunternehmen mit Sitz in New York und Paris.
French Translation:  Acme est une société technologique basée à New York et à Paris.
